<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/DP-SGD/wine_DP_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import copy
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager

In [4]:
!pip install opacus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [6]:
#preparing dataset
wine = load_wine()
# x is the carachteristics and y the labels
x = wine.data
y = wine.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Padronizar os dados para média zero e variância unitária, melhora o treinamento
#transforming trains and tests munpy arrays to tensors
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)  # Padroniza os dados de treino
x_test = scaler.transform(x_test)  # Padroniza os dados de teste (usando os mesmos parâmetros do treino)

# Converter para tensores do PyTorch
x_train= torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [7]:
#defining model
#herda de torch.nn.Module
class MLP(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    #chama superclasse
    super(MLP, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)
    #define o comportamento da rede neural
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x



In [ ]:
print(x_train.size())
print(x_test.size())

torch.Size([142, 13])
torch.Size([36, 13])


In [19]:
train_dataset = TensorDataset(x_train, y_train)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

privacy_engine = PrivacyEngine()
model = MLP(input_size=x_train.size()[1], hidden_size=128, output_size=3)
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optim.Adam(model.parameters(), lr=0.009, weight_decay=1e-5),
    data_loader=train_loader,
    target_epsilon=1,
    target_delta = 1e-5,
    epochs=150,
    poisson_sampling=False,
    max_grad_norm=0.5
)


In [22]:
#defining loss function
criterion = torch.nn.CrossEntropyLoss()
#defining optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.009)

epochs = 100

for epoch in range(epochs):
  model.train()
  #always remember this!!
  for x_batch, y_batch in train_loader:
    outputs = model(x_batch)
    loss = criterion(outputs, y_batch)

    #calculating gradients
    loss.backward()
    #updating weights
    optimizer.step()
  if (epoch + 1) % 10 == 0:
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_batch).sum().item() / len(y_batch)
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f} Accuracy: {accuracy * 100:.2f}%')


Epoch [10/100], Loss: 0.0526 Accuracy: 100.00%
Epoch [20/100], Loss: 0.3289 Accuracy: 83.33%
Epoch [30/100], Loss: 0.1194 Accuracy: 100.00%
Epoch [40/100], Loss: 0.7567 Accuracy: 83.33%
Epoch [50/100], Loss: 1.0011 Accuracy: 83.33%
Epoch [60/100], Loss: 0.0041 Accuracy: 100.00%
Epoch [70/100], Loss: 0.5285 Accuracy: 66.67%
Epoch [80/100], Loss: 0.9730 Accuracy: 50.00%
Epoch [90/100], Loss: 0.1936 Accuracy: 100.00%
Epoch [100/100], Loss: 0.0036 Accuracy: 100.00%


In [21]:
#evaluating model
#no need to calculate gradient in evaluation
model.eval()
with torch.no_grad():
  outputs = model(x_test)
  _, predicted = torch.max(outputs.data, 1)
  accuracy = (predicted == y_test).sum().item() / len(y_test)
  print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 100.00%
